In [1]:
import pandas as pd


In [11]:
data = pd.read_csv('TPSS_region_OD3.csv', encoding='cp949', sep=",")
data.head()

,Date,OA,OB,DA,DB,population
0,20230920,성동구,성수2가1동,은평구,대조동,3
1,20230920,성동구,성수2가1동,은평구,불광1동,2
2,20230920,성동구,성수2가1동,은평구,불광2동,1
3,20230920,성동구,성수2가1동,은평구,수색동,2
4,20230920,성동구,성수2가1동,은평구,신사1동,3


In [13]:
data['OA'].unique()

array(['성동구', '성북구', '송파구', '수원시영통구', '시흥시', '안산시상록구', '안양시동안구', '안양시만안구',
       '양주시', '양천구', '양평군', '연수구', '영등포구', '용산구', '용인시기흥구', '용인시수지구',
       '용인시처인구', '은평구', '의정부시', '종로구', '중구', '중랑구', '파주시', '성남시분당구',
       '성남시수정구', '성남시중원구', '수원시장안구', '여주시', '하남시', '수원시팔달구', '안산시단원구',
       '이천시', '서초구', '평택시', '화성시', '영통구', '의왕시', '오산시', '수원시권선구', '강동구',
       '강북구', '강서구', '고양시덕양구', '고양시일산동구', '고양시일산서구', '과천시', '관악구', '광명시',
       '광주시', '광진구', '구로구', '구리시', '군포시', '금천구', '김포시', '남구', '남양주시',
       '노원구', '도봉구', '동대문구', '동두천시', '동작구', '강남구', '마포구', '남동구', '부천시',
       '가평군', '부평구', '서구', '서대문구', '계양구', '서울시외', '동구'], dtype=object)

In [15]:
data['DA'].unique()

array(['은평구', '의정부시', '종로구', '중구', '중랑구', '파주시', '하남시', '가평군', '강남구',
       '강동구', '강북구', '강서구', '계양구', '고양시덕양구', '서울시외', '서초구', '성남시분당구',
       '성남시수정구', '성남시중원구', '성동구', '성북구', '송파구', '수원시팔달구', '안양시동안구',
       '안양시만안구', '양주시', '양천구', '양평군', '연수구', '영등포구', '영통구', '용산구',
       '용인시기흥구', '용인시수지구', '구로구', '구리시', '군포시', '금천구', '김포시', '남구', '남동구',
       '남양주시', '노원구', '도봉구', '동구', '동대문구', '동두천시', '동작구', '마포구', '부천시',
       '부평구', '서구', '서대문구', '고양시일산동구', '고양시일산서구', '과천시', '관악구', '광명시',
       '광주시', '광진구', '시흥시', '안산시단원구', '안산시상록구', '화성시', '수원시권선구', '수원시영통구',
       '수원시장안구', '여주시', '오산시', '용인시처인구', '이천시', '의왕시', '평택시'],
      dtype=object)

In [45]:
filterdDf =data[(data['OB']!=data['DB'])&(data['Date']==20230920)&(data['population']>100)].sort_values('population', ascending= False)

output =filterdDf.loc[:,['OB','DB','population']]

# A에서 B로 이동하는 데이터 복사
output1 = output.copy()

# 열 이름 바꾸기
# df1.columns = ['Location_1', 'Location_2', 'Value']

# B에서 A로 이동하는 데이터 복사
output2 = output.copy()

# 열 이름 바꾸기
output2.columns = ['DB', 'OB', 'population']

In [48]:
# Score 계산

output['similarity'] = output['population']/output.max()['population']

5990

In [92]:
# A에서 B로 이동하는 데이터 복사
output1 = output.copy()

# B에서 A로 이동하는 데이터 복사
output2 = output.copy()

# 열 이름 바꾸기
output2.columns = ['DB', 'OB', 'population']

# 두 데이터프레임 병합
merged_df = pd.concat([output1, output2])

# 동일한 출발지와 도착지를 가진 데이터 합치기
result_df = merged_df.groupby(['DB', 'OB']).sum().reset_index()

# (Location_1=지역1 & Location_2=지역3)와 (Location_1=지역3 & Location_2=지역1)을 같은 것으로 처리
result_df['Unique_DB'] = result_df[['DB', 'OB']].min(axis=1)
result_df['Unique_OB'] = result_df[['DB', 'OB']].max(axis=1)

# 중복된 데이터 제거
result_df = result_df.drop_duplicates(subset=['Unique_DB', 'Unique_OB'])

# 불필요한 열 제거
result_df = result_df.drop(columns=['Unique_DB', 'Unique_OB'])




In [102]:
result_df['similarity'] = result_df['population']/result_df.max()['population']

In [103]:
result_df.sort_values('similarity', ascending= False)

,DB,OB,population,similarity
10730,서울시외,종로1.2.3.4가동,8704,1.000000
5971,명동,서교동,8271,0.950253
11259,서초2동,잠실6동,8085,0.928883
19933,종로1.2.3.4가동,회현동,8005,0.919692
6113,명동,회현동,7650,0.878906
...,...,...,...,...
12341,성현동,신림동,101,0.011604
6014,명동,쌍문2동,101,0.011604
1603,교남동,여의동,101,0.011604
15603,양평2동,영등포본동,101,0.011604


In [106]:
output =result_df.loc[:,['OB','DB','similarity']].sort_values('similarity', ascending= False).reset_index()

In [108]:
output.to_csv('sample3.csv')

In [107]:
output

,index,OB,DB,similarity
0,10730,종로1.2.3.4가동,서울시외,1.000000
1,5971,서교동,명동,0.950253
2,11259,잠실6동,서초2동,0.928883
3,19933,회현동,종로1.2.3.4가동,0.919692
4,6113,회현동,명동,0.878906
...,...,...,...,...
11748,12341,신림동,성현동,0.011604
11749,6014,쌍문2동,명동,0.011604
11750,1603,여의동,교남동,0.011604
11751,15603,영등포본동,양평2동,0.011604
